In [1]:
import cv2 as cv
import matplotlib.pyplot as plt
import os

In [2]:
def getMedian(lst):
	temp = sorted(lst)
	if (len(lst) % 2 == 0):
		return sum([temp[((len(temp) // 2) - 1)], temp[(len(temp) // 2)]]) / 2, ((len(temp) // 2) - 1)
	else:
		return temp[len(temp) // 2], (len(temp) // 2)

In [3]:
def findOutliers(lst):
	temp = sorted(lst)
	med, loc = getMedian(lst)
	lower = 0;
	upper = 0;
	if (len(lst) % 2 == 0):
		lower = getMedian(temp[:loc + 1])[0]
		upper = getMedian(temp[loc + 1:])[0]
	else:
		lower = getMedian(temp[:loc])[0]
		upper = getMedian(temp[loc + 1:])[0]

	range = upper - lower
	min = lower - 1.5*range
	max = upper + 1.5*range

	"""print(f"L outlier: {min}")
	print(f"L        : {lower}")
	print(f"Median   : {med}")
	print(f"U        : {upper}")
	print(f"U outlier: {max}")"""

	# Find outliers
	outliers = []
	for i, val in enumerate(lst):
		if (val < min or val > max):
			outliers.append(i)
		
	return outliers


In [68]:
def processImage(fileName, imageType):
	# print(fileName)
	img = cv.imread(fileName, cv.IMREAD_COLOR)
	# print(img.shape)
	finalImage = img[:, :, ::-1]
	# plt.imshow(finalImage)

	# Change image type if needed
	if (imageType == 'hsv'):
		finalImage = cv.cvtColor(finalImage, cv.COLOR_RGB2HSV)
		# plt.imshow(finalImage)

	# data
	data = [[] for _ in range(finalImage.shape[2])]

	# Sort the data
	for r in range(finalImage.shape[0]):
		for c in range(finalImage.shape[1]):
			for i in range(finalImage.shape[2]):
				data[i % finalImage.shape[2]].append(finalImage[r][c][i % finalImage.shape[2]])
	# print('finshed sorting data')

	# Remove outliers
	indexes = []
	for i in range(finalImage.shape[2]):
		badValues = findOutliers(data[i])
		if (len(badValues) > 0):
			indexes.extend(badValues)
	outliers = set(indexes)
	outliers = sorted(list(outliers), reverse = True)
	# print('found outliers')

	for i in range(finalImage.shape[2]):
		# print('removing from shape index', i)
		for j, index in enumerate(outliers):
			del data[i][index]
			# print('deleted ', j + 1, ' out of ', len(outliers), end='\r')
	# print('removed outliers')

	# Display the data
	# for i in range(finalImage.shape[2]):
	# 	print(f"{imageType[0].upper()} Min: {min(data[i])}")
	# 	print(f"{imageType[0].upper()} Avg: {sum(data[i]) / len(data[i]):.1f}")
	# 	print(f"{imageType[0].upper()} Med: {getMedian(data[i])[0]}")
	# 	print(f"{imageType[0].upper()} Max: {max(data[i])}")
	# 	print()
	# print('~~~~~~~~~~~~~~~~~~~~~~~')
	return data

In [72]:
data = [[] for _ in range(3)]

for filename in os.listdir('Images/Color-Calibration-JPG/Test-Set'):
    f = os.path.join('Images/Color-Calibration-JPG/Test-Set', filename)
    # checking if it is a file
    if os.path.isfile(f):
        if (os.path.splitext(f)[1] == '.jpg'):
            info = processImage(f, 'rgb')
            for i in range(len(info)):
                data[i].extend(info[i])

for i, color in enumerate(['R', 'G', 'B']):
    print(f"{color} Min: {min(data[i])}")
    print(f"{color} Avg: {sum(data[i]) / len(data[i]):.1f}")
    print(f"{color} Med: {getMedian(data[i])[0]}")
    print(f"{color} Max: {max(data[i])}")
    print()
        


R Min: 149
R Avg: 199.6
R Med: 197.0
R Max: 246

G Min: 156
G Avg: 206.7
G Med: 205.0
G Max: 254

B Min: 43
B Avg: 81.0
B Med: 78.0
B Max: 129



In [51]:
def translate(value, leftMin, leftMax, rightMin, rightMax):
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

In [73]:
print(translate(31, 0, 255, 0, 360))
print(translate(161, 0, 255, 0, 100))
print(translate(179, 0, 255, 0, 100))

43.76470588235294
63.13725490196078
70.19607843137254
